In [73]:
import os
import re
from pprint import pprint

path = '/home/legha/bin/Version5_ev1738/Dragon/Linux_x86_64/msca'
pprint(os.listdir(path))

import os.path as op
with open(op.join(path, 'CGN_PIN_A.result')) as f:
    long_str = f.read()

len(long_str)
print(len(long_str.splitlines()))

['assertS.c2m',
 'assertV.c2m',
 '_BURN_rowland.txt',
 'CGN_PIN_A.result',
 'code',
 'DLIB_295',
 '_EDIT_rowland.txt',
 '_LIB_rowland.txt',
 'PIN_FIG.ps',
 'mydata.x2m']
305336


In [86]:
import re

end='.*\n'
line=[m.end() for m in re.finditer(end, long_str)]

pattern = '>\|(BURNUP step \d+.*?)(?=>\|BURNUP step|>\|CGN_PIN_A completed)'
match=re.compile(pattern, re.MULTILINE|re.DOTALL)
for m in re.finditer(match, long_str):
    lineno = next(i+1 for i in range(len(line)) if line[i]>m.start(1))
    endno = lineno + len(m.group().splitlines()) - 1
    text = m.group()[:70]
    print(f'lineno start:{lineno: <6} end:{endno: <6} text: {text}')

lineno start:8117   end:13070  text: >|BURNUP step 1 between  0.000000e+00 and  7.692308e-01 day:          
lineno start:13071  end:18017  text: >|BURNUP step 2 between  7.692308e-01 and  1.282051e+00 day:          
lineno start:18018  end:22963  text: >|BURNUP step 3 between  1.282051e+00 and  1.923077e+00 day:          
lineno start:22964  end:27910  text: >|BURNUP step 4 between  1.923077e+00 and  3.846154e+00 day:          
lineno start:27911  end:32863  text: >|BURNUP step 5 between  3.846154e+00 and  6.410256e+00 day:          
lineno start:32864  end:37816  text: >|BURNUP step 6 between  6.410256e+00 and  1.282051e+01 day:          
lineno start:37817  end:42769  text: >|BURNUP step 7 between  1.282051e+01 and  1.923077e+01 day:          
lineno start:42770  end:47722  text: >|BURNUP step 8 between  1.923077e+01 and  2.564103e+01 day:          
lineno start:47723  end:52677  text: >|BURNUP step 9 between  2.564103e+01 and  5.128205e+01 day:          
lineno start:52678  end:5763

['DLIB_295',
 'CGN_PIN_A_short.result',
 'PIN_FIG.ps',
 '_BURN_rowland.txt',
 '_LIB_rowland.txt',
 '_EDIT_rowland.txt']
22886


In [92]:
import os
import re
from pprint import pprint

path = '/home/legha/bin/Version5_ev1738/Dragon/msca/output_short'
pprint(os.listdir(path))

import os.path as op
with open(op.join(path, 'CGN_PIN_A_short.result')) as f:
    short_str = f.read()
    
print(len(short_str.splitlines()))

def parse_echo(s : str)-> None:
    end='.*\n'
    line=[m.end() for m in re.finditer(end, s)]

    pattern = '>\|(.*?)(?=\|>\d+)'
    match=re.compile(pattern, re.MULTILINE|re.DOTALL)
    for m in re.finditer(match, s):
        lineno = next(i+1 for i in range(len(line)) if line[i]>m.start(1))
        endno = lineno + len(m.group().splitlines()) - 1
        text = m.group()
        print(f'start:{lineno: <6} end:{endno: <6} text: {text}')
        
parse_echo(short_str)

['DLIB_295',
 'CGN_PIN_A_short.result',
 'PIN_FIG.ps',
 '_BURN_rowland.txt',
 '_LIB_rowland.txt',
 '_EDIT_rowland.txt']
22886
start:7998   end:7998   text: >|STARTING WHILE                                                          
start:8006   end:8006   text: >|BURNUP step 1 between  0.000000e+00 and  7.692308e-01 day:              
start:8335   end:8335   text: >|StepAuto:  3.000000e+01 MWj/t                                           
start:8336   end:8336   text: >|Self-shielding calculation 1 at  7.692308e-01 DAY:                      
start:9565   end:9565   text: >|step2=  3.000000e+01 evoend=  7.692308e-01                              
start:12914  end:12914  text: >|BURNUP:   3.000000e+01   <-> K-INFINITY :   1.055019e+00                
start:12953  end:12953  text: >|THIS IS BEFORE STEP increase  1                                         
start:12954  end:12954  text: >|THIS IS AFTER STEP increase  2                                          
start:12955  end:12955  text: >|ST

In [65]:
>>> width=10
>>> string = 'sss'
>>> f'{string: <10}'

'sss       '

In [ ]:
parse_echo(long_str)